# Next Word Predictor using Pytorch | LSTM using PyTorch

## Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Downloading punkt and punk_tab to run nltk     

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Loading and Preprocessing the Data

### Defining Dataset

In [4]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


### Tokenize the text data

In [5]:
tokens = word_tokenize(document.lower())

In [6]:
vocab = {'<unk>':0}

for token in Counter(tokens).keys():
    if token not in vocab:
        vocab[token] = len(vocab)

In [7]:
len(vocab)

289

### Extract Sentence from the text data

In [8]:
input_sentence = document.split('\n')

In [9]:
len(input_sentence)

78

In [10]:
input_sentence[:5]

['About the Program',
 'What is the course fee for  Data Science Mentorship Program (DSMP 2023)',
 'The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.',
 'What is the total duration of the course?',
 'The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)']

### Converting words to integer values

In [ ]:
def words_to_indices(sentence, vocab):

    numerical_sentence = []

    for token in sentence:
        if token in vocab:
            numerical_sentence.append(vocab[token])
        else:
            numerical_sentence.append(vocab['<unk>'])

    return numerical_sentence

In [12]:
input_num_sentence = []

for sentence in input_sentence:
    input_num_sentence.append(words_to_indices(word_tokenize(sentence.lower()), vocab))

In [13]:
input_sentence[:5]

['About the Program',
 'What is the course fee for  Data Science Mentorship Program (DSMP 2023)',
 'The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.',
 'What is the total duration of the course?',
 'The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)']

In [14]:
for i in range(0,5):
    print(input_sentence[i])
    print(input_num_sentence[i])

About the Program
[1, 2, 3]
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
[4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14, 15]
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27, 28, 29, 30]
What is the total duration of the course?
[4, 5, 2, 31, 32, 27, 2, 6, 33]
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
[2, 31, 32, 27, 2, 6, 5, 34, 35, 30, 36, 2, 31, 6, 7, 37, 38, 39, 34, 40, 28, 41, 12, 42, 30, 15]


### Creating Training Sequences

In [15]:
training_sequences = []

for num_sentence in input_num_sentence:

    for i in range(1, len(num_sentence)):
        training_sequences.append(num_sentence[:i+1])

In [16]:
training_sequences[:5]

[[1, 2], [1, 2, 3], [4, 5], [4, 5, 2], [4, 5, 2, 6]]

### Length of all Training Sequences

In [17]:
length_sequence = []
for sequence in training_sequences:
    length_sequence.append(len(sequence))

In [18]:
max(length_sequence)

62

### Add Padding to Training Sequences

In [19]:
pad_train_sequences = []
for sequence in training_sequences:
    pad_train_sequences.append([0]*(max(length_sequence) - len(sequence)) + sequence)

In [20]:
len(pad_train_sequences[0])

62

In [21]:
len(pad_train_sequences)

942

### Converting Padding Sequences to Tensors

In [22]:
pad_train_sequences = torch.tensor(pad_train_sequences, dtype=torch.long)

In [23]:
pad_train_sequences

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   0,   4,   5],
        ...,
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287],
        [  0,   0,   0,  ..., 286, 287, 288]])

### Creating X and y Tensors

In [24]:
X = pad_train_sequences[:,:-1]
y = pad_train_sequences[:,-1]

In [25]:
X

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   0,   0,   4],
        ...,
        [  0,   0,   0,  ...,   0, 285, 176],
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287]])

In [26]:
y

tensor([  2,   3,   5,   2,   6,   7,   8,   9,  10,  11,   3,  12,  13,  14,
         15,   6,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  18,  26,
         27,  28,  29,  30,   5,   2,  31,  32,  27,   2,   6,  33,  31,  32,
         27,   2,   6,   5,  34,  35,  30,  36,   2,  31,   6,   7,  37,  38,
         39,  34,  40,  28,  41,  12,  42,  30,  15,   5,   2,  43,  27,   2,
         11,   3,  33,  45,  46,  47,   2,  48,  49,  50,  52,  53,   8,   9,
         10,  54,   8,   9,  10,   8,  57,  58,  60,  59,  64,  65,  66,   2,
         67,  43,  68,  69,  70,  50,  71,  72,  58,  73,  74,  46,  17,  75,
         27,  76,   3,  33,  78,  74,  73,  72,  58,  79,  80,  81,  17,  75,
         27,  76,   3,  82,  83,  84,  30,  85,  86,  87,  17,  88,  89,  33,
         45,  86,  90,  17,  91,  27,   2,  89,  33,  93,  94,  95,  80,  96,
         78,  36,  97,  85,  22,  87,  17,  89,  22,  65,  98,  99,  73, 100,
          2,  91,  30,  65,  86, 101,   2, 102, 103,  33,  76, 1

## Creating Batches of Data

### Creating Custom Dataset

In [27]:
class CustomDataset(Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]


In [28]:
dataset = CustomDataset(X,y)

In [29]:
len(dataset)

942

In [30]:
dataset[1]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2]),
 tensor(3))

### Creating Dataloader of Dataset Object

In [31]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## LSTM Model

### Architecture

In [38]:
class Model(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=100
        )
        self.lstm = nn.LSTM(
            input_size=100, hidden_size=150, batch_first=True
        )
        self.linear = nn.Linear(
            in_features=150, out_features=vocab_size
        )

    def forward(self, x):
        embedded = self.embedding(x)
        intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
        output = self.linear(final_hidden_state.squeeze(0))

        return output

### Model Object

In [39]:
model = Model(len(vocab)).to(device)

### Epochs and Learning Rate

In [40]:
epochs = 50
learning_rate = 0.001

### Loss and Optimizer

In [41]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Training Loop

In [42]:
for epoch in range(epochs):

    total_loss = 0

    for batch_input, batch_output in dataloader:
        
        batch_input, batch_output = batch_input.to(device), batch_output.to(device)

        optimizer.zero_grad()

        y_pred = model(batch_input)

        loss = loss_function(y_pred,batch_output)

        loss.backward()

        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch: {epoch+1}, Loss: {total_loss/len(dataloader)}")

Epoch: 1, Loss: 5.537179787953694
Epoch: 2, Loss: 4.908297920227051
Epoch: 3, Loss: 4.503918226559957
Epoch: 4, Loss: 4.112590003013611
Epoch: 5, Loss: 3.6923169692357383
Epoch: 6, Loss: 3.3014938672383627
Epoch: 7, Loss: 2.933215037981669
Epoch: 8, Loss: 2.6178515752156577
Epoch: 9, Loss: 2.3021066347757975
Epoch: 10, Loss: 2.042221796512604
Epoch: 11, Loss: 1.7937581141789753
Epoch: 12, Loss: 1.5837894956270853
Epoch: 13, Loss: 1.3851348360379536
Epoch: 14, Loss: 1.2238182783126832
Epoch: 15, Loss: 1.0759554743766784
Epoch: 16, Loss: 0.9542486409346262
Epoch: 17, Loss: 0.8287358423074086
Epoch: 18, Loss: 0.7408446510632832
Epoch: 19, Loss: 0.6598354538281759
Epoch: 20, Loss: 0.5920572300752004
Epoch: 21, Loss: 0.5278891861438751
Epoch: 22, Loss: 0.47794891595840455
Epoch: 23, Loss: 0.4312828858693441
Epoch: 24, Loss: 0.39618364671866096
Epoch: 25, Loss: 0.36564973096052805
Epoch: 26, Loss: 0.3424068649609884
Epoch: 27, Loss: 0.32036031434933343
Epoch: 28, Loss: 0.30299591074387233
Ep

### Prediction Function

In [ ]:
def predict(model, vocab, text):

    # convert text to token
    token = word_tokenize(text)
    
    # convert token to numbers
    num_token = words_to_indices(token, vocab)

    # add padding
    pad_token = [0] * (max(length_sequence) - len(num_token)) + num_token

    # convert to tensor
    tensor_text = torch.tensor(pad_token, dtype=torch.long).unsqueeze(0).to(device)

    # send to model
    output = model(tensor_text)

    # output index
    val, index = torch.max(output, dim=1)

    # vocab index
    output_word = list(vocab.keys())[index]
    
    # merge with text
    final_text = text + ' ' + output_word

    return final_text

In [91]:
input_text = 'What is the course fee'
num_token = 10

for i in range(num_token):
    input_text = predict(model, vocab, input_text)
    print(f"{i+1}. {input_text}")
    

1. What is the course fee for
2. What is the course fee for data
3. What is the course fee for data science
4. What is the course fee for data science mentorship
5. What is the course fee for data science mentorship program
6. What is the course fee for data science mentorship program (
7. What is the course fee for data science mentorship program ( dsmp
8. What is the course fee for data science mentorship program ( dsmp 2023
9. What is the course fee for data science mentorship program ( dsmp 2023 )
10. What is the course fee for data science mentorship program ( dsmp 2023 ) you
